In [ ]:
!pip install mlflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Dropout, GRU, Dense, TimeDistributed, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import mlflow
import mlflow.keras
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
def build_model(input_shape=(30, 21), output_steps=7, output_features=6):
    inputs = Input(shape=input_shape)
    
    x = Conv1D(64, kernel_size=3, activation='relu', padding='causal')(inputs)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='causal')(x)
    x = Dropout(0.2)(x)

    x = GRU(64, return_sequences=True)(x)
    x = LayerNormalization()(x)
    x = GRU(64, return_sequences=True)(x)
    x = LayerNormalization()(x)

    x = TimeDistributed(Dense(output_features))(x)
    outputs = x[:, -output_steps:, :]

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
    
    return model


In [ ]:
def train_and_save_model(
    X_path="/kaggle/input/weather/X.npy",
    y_path="/kaggle/input/weather/y.npy",
    save_path="/kaggle/working/weather_model.keras",
    mlflow_tracking_uri='/kaggle/working/mlflow',
    mlflow_experiment="Weather_Forecast_Improved",
    epochs=50,
    batch_size=32,
    patience=7
):
    X = np.load(X_path).astype(np.float32)
    y = np.load(y_path).astype(np.float32)

    input_shape = X.shape[1:]
    output_steps = y.shape[1]
    output_features = y.shape[2]

    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(mlflow_experiment)

    with mlflow.start_run(run_name="Improved_Model_Training"):
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("input_shape", input_shape)
        mlflow.log_param("output_steps", output_steps)
        mlflow.log_param("output_features", output_features)

        model = build_model(input_shape, output_steps, output_features)

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
            ModelCheckpoint(save_path, monitor='val_loss', save_best_only=True)
        ]

        history = model.fit(
            X, y,
            validation_split=0.2,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )

        for epoch in range(len(history.history["loss"])):
            mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
            mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
            mlflow.log_metric("train_mae", history.history["mae"][epoch], step=epoch)
            mlflow.log_metric("val_mae", history.history["val_mae"][epoch], step=epoch)

        mlflow.keras.log_model(model, artifact_path="model")
        mlflow.log_artifact(save_path)

        print(f"✅ Model trained and saved to {save_path}")


In [ ]:
def evaluate_model(
    model_path="/kaggle/working/weather_model.keras",
    X_path="/kaggle/input/weather/X.npy",
    y_path="/kaggle/input/weather/y.npy"
):
    X = np.load(X_path).astype(np.float32)
    y = np.load(y_path).astype(np.float32)

    _, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = load_model(model_path)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test.reshape(-1, y.shape[-1]), y_pred.reshape(-1, y.shape[-1]))
    mae = mean_absolute_error(y_test.reshape(-1, y.shape[-1]), y_pred.reshape(-1, y.shape[-1]))

    print(f"\n📊 Test MSE: {mse:.4f}")
    print(f"📊 Test MAE: {mae:.4f}")
    print("\n🔍 MAE per feature:")
    feature_mae = np.mean(np.abs(y_pred - y_test), axis=(0, 1))
    for i, f_mae in enumerate(feature_mae):
        print(f"Feature {i}: MAE = {f_mae:.4f}")

    return X_test, y_test, y_pred


In [ ]:
def plot_prediction(y_true, y_pred, sample_idx=0):
    steps = y_true.shape[1]
    features = y_true.shape[2]

    plt.figure(figsize=(16, 8))
    for feature in range(features):
        plt.plot(range(steps), y_true[sample_idx, :, feature], label=f"True Feature {feature}")
        plt.plot(range(steps), y_pred[sample_idx, :, feature], linestyle='--', label=f"Pred Feature {feature}")
    plt.title(f"Prediction vs Ground Truth (Sample {sample_idx})")
    plt.xlabel("Forecast Time Step")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
train_and_save_model()
X_test, y_test, y_pred = evaluate_model()
plot_prediction(y_test, y_pred, sample_idx=0)

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# --- Paths ---
X_path = "/kaggle/input/weather/X.npy"
y_path = "/kaggle/input/weather/y.npy"
model_path = "/kaggle/working/weather_model.keras"

# Load model and data
model = load_model(model_path)
X = np.load(X_path).astype(np.float32)
y = np.load(y_path).astype(np.float32)

# Select a sample index (within range)
sample_idx = 2
x_sample = X[sample_idx:sample_idx+1]  # shape: (1, 30, 21)
y_true = y[sample_idx]                 # shape: (7, 6)

# Predict
y_pred = model.predict(x_sample)[0]    # shape: (7, 6)

# Targets
targets = ['tavg', 'tmin', 'tmax', 'wspd', 'prcp', 'snow']

# Build comparison table
df = pd.DataFrame(y_true, columns=[f"True_{t}" for t in targets])
for i, t in enumerate(targets):
    df[f"Pred_{t}"] = y_pred[:, i]

# Round for readability
df = df.round(2)

# Show result
print(f"\n📊 Prediction vs True Values for Sample {sample_idx}:\n")
print(df)


In [ ]:
!zip -r /kaggle/working/mlflow.zip /kaggle/working/mlflow